# **Homework 03: Machine Learning**
## "Going Down the EECS Stack" Decal, Fall 2023

In lecture, we talked about a variety of common machine learning methods, their advantages and disadvantages, and how they can be utilized to help us make sense of data. We specifically talked about two broad types of ML: supervised and unsupervised (put picture and define further)

In this assignment, we'll be introducing another broad category of ML known as **Reinforcement Learning (RL)**. RL is a training method that rewards desired actions and punishes undesired actions to achieve some goal.

Before we get started, in case you're not familiar with Jupyter Notebooks, you can read more about them [here](https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html). At several points you'll be asked to run the code within a cell. To do so, just click anywhere in the cell and type "shift + enter."

It's not too different from how a dog learns to do tricks. If Fido rolls over when told, he gets a treat and is more likely to roll over in the future. But if he decides to sit when told to roll over, he won't get a treat (negative reinforcement), and won't sit in the future when told to roll over. 

In this scenario, Fido is what's known as an **"intelligent agent."** He's learning what actions to take (rolling over, sitting, etc.) based on a **reward function** (whether he gets a treat or not). More generally, the agent performs some action that makes a change to the environemnt to reveal a new state. A reward function is calculated based on this new state, and the agent considers both the reward and state that resulted from the previous action to decide if they should continue to perform that action or try something new. 

![rl_diagram](rl_diagram.png)

RL can essentially be broken down into 6 simple steps:
1. Observe the environment
2. Decide how to act using some strategy
3. Act accordingly
4. Receive a reward or penalty
5. Learn from the experiences and refine strategy
6. Iterate until an optimal strategy is found

### Uses

Reinforcement Learning is the type of machine learning that most closely mimics how humans learn, which offers some unique advantages. It's most commonly utilized in fields like robotics, natural language processing (NLP), and marketing. 

## **Q Learning**

In this assignment we'll specifically be discussing an algorithm known as Q learning. Q Learning is known as an *Off-Policy* algorithm, meaning it doesnn't follow some pre-determined policy, and instead starts off by taking random actions to eventually learn the best action to take in each state. 

While Fido uses his brain to store information about treats and rolling over, we will take a more minimalist approach and us a simple 2D array known as a **Q Table**. This table keeps track of how favorable an action-state combination is in terms of rewards. Each column corresponds to a different action, and each row corresponds to a different state of the environment. As you can imagine, that means Q Learning is best suited for situations in which the action and environment spaces are both discrete and small, so the Q Table doesn't grow too large. 

In fact, another form of RL known as Deep Q Learning operates by the same premise as Tabular Q Learning but replaces the table with a neural network to better deal with more complex actions and environments.


### **Exploration-Exploitation Tradeoff**

As mentioned above, the Q Learning agent begins by taking random actions so it can start populating the Q Table. This process is known as **exploration**, because it allows the algorithm to try out and learn about different actions. After a while, however, we want the agent to start **exploiting** the information we have in the table to make the best decision. But how do we know when to explore and when to exploit?

We use what's called the **epsilon ($\epsilon$) greedy model**. We start with an epsilon such that 0 <= $\epsilon$ <= 1. We then generate a random number between 0 and 1. If the value is less than $\epsilon$, we explore and choose a random action, but if the value is greater than $\epsilon$, we exploit and pick the best possible action from the row in the Q Table corresponding to the current environment state (hence the term "greedy"). 

**For what value of $\epsilon$ will the algorithm only explore? For what value will it only exploit?**

**Answer:** 

As we continue to train the model, we want to start decreasing the value of $\epsilon$ so that the model starts exploiting more, given that it should gain more useful information as time goes on. As we work on our example, we'll see how this can be implemented. 

### **Q Values (Bellman Equation)**

The actual values stored in the Q Table aren't just the reward values, but they are dependent on them. The exact equation is:

![bellman_eq.png](bellman_eq.png)

You won't be expected to fully understand how the equation works in detail (feel free to look it up if you're interested), but the main things to notice are that the Q value at the given state action pair (S<sub>t</sub>, A<sub>t</sub>) is being updated by not just the reward (R<sub>t+1</sub>) associated with performing the action, but also the max Q value of the future state (S<sub>t+1</sub>) that would result from taking this action. In that sense, we're not just looking at how favorable the action would be in the current state, but in the future state as well. 

Now that we've gotten an introduction to the basics of Q Learning, let's implement our own algorithm!

# **Example: Taxi**

In this scenario, we will be using Q Learning to teach a taxi driver how to successfully pick up and drop off a passenger. The passenger will randomly start each episode in one of 4 designated pickup/dropoff zones (represented by the colored squares), and the taxi will randomly start in one of those squares as well. A successful episode is one in which the taxi picks up the passenger and drops them off at their desired square, which is also randomly decided and represented by a building.

![env_pic.png](env_pic.png)

Note that the environment looks wider than 5 columns, but there are dividers where the taxi can't go, so there are only 5 possible x positions and 5 possible y positions, for a total of 25 positions.

**What is the "intelligent agent" in this scenario?**

Answer: 

We will be using a Python module greated by OpenAI called Gym to help create the environment for our scenario. Taxi is just one of the many predefined environments that Gym has. **You don't need to worry about all the specifics of Gym or how we are creating the environment in the following code block, but here are some details you should understand:**

### Action Space and Environment Space:

The action space is the set of all actions that the agent can take. In our case there are **6**:
- 0: Move down
- 1: Move up
- 2: Move right
- 3: Move left
- 4: Pick up passenger
- 5: Drop off passenger

The environment space is the set of all possible states of the environment. In this case there are **500**: there are 25 possible locations for the taxi on the 5x5 grid, 5 potential passenger locations (a pickup/drop off zone or in the taxi itself), and 4 possible destinations.

*Each state is uniquely represented by an integer from 0 to 499, if you're curious how it is calculated, [click here](https://gymnasium.farama.org/environments/toy_text/taxi/)*

#### **Env.step()**
The step function is what actually applies the action to the environment, so in our case that's moving the taxi. It returns the new state, the reward, and whether or not the passenger was successfully dropped off ("done" variable). 

#### **Env.reset()**
Chooses a random starting point for the taxi and passenger, as well as a random destination. Returns this new state.

#### **Reward**
The reward function is defined for this environment by default as follows
-1 for every step
+20 for successfully delivering the passenger
-10 for every incorrect or illegal pickup/dropoff

Some other terminology...

**Episode**: one run-through of the algorithm. In this case, it's one attempt of the taxi to successfully deliver the passenger. We end an episode once either the passenger is succssfully delivered, or we reach a threshold value for number of epochs in that episode.

**Epoch**: one step in the episode. Each epoch in this scenario corresponds to a move by the taxi.

Now let's visualize a runthrough of one episode where we are just randomly sampling actions to see what the taxi looks like in action. Note that although a reward function is being calculated, the agent is not learning from it in any way, which is why this taxi has no idea what it's doing.

First we'll have to install all the Python packages we'll be using.

In [ ]:
!pip3 install gym numpy matplotlib IPython gym[toy_text]

In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import random
from IPython.display import clear_output
from time import sleep
from matplotlib import animation
from utils import run_animation, store_episode_as_gif

"""Simulation with random agent"""
env = gym.make("Taxi-v3", render_mode="rgb_array")

epoch = 0
num_failed_dropoffs = 0
experience_buffer = []
cum_reward = 0

done = False

state, _ = env.reset()

while not done and epoch < 100:
    # Sample random action
    "Action selection without action mask"
    action = env.action_space.sample()

    "Action selection with action mask"
    #action = env.action_space.sample(env.action_mask(state))

    state, reward, done, _, _ = env.step(action)
    cum_reward += reward

    # Store experience in dictionary
    experience_buffer.append({
        "frame": env.render(),
        "episode": 1,
        "epoch": epoch,
        "state": state,
        "action": action,
        "reward": cum_reward,
        }
    )

    if reward == -10:
        num_failed_dropoffs += 1

    epoch += 1

# Run animation and print console output
run_animation(experience_buffer, True)

print("# epochs: {}".format(epoch))
print("# failed drop-offs: {}".format(num_failed_dropoffs))

Now that we've set up the environment, let's help our passenger reach their destination through Q Learning!

## **Q Table**
Here is what our Q Table will look like for our scenario:

![qtable_pic](qtable_pic.png)

**What are the dimensions of our Q Table?**

Answer: 

We must start by initializing the Q Table with zeroes since we don't have any information to start with.

In [ ]:
state_size = env.observation_space.n  # total number of states (S)
action_size = env.action_space.n      # total number of actions (A)

# initialize a qtable with 0's for all Q-values
qtable = np.zeros((state_size, action_size))

## **Hyperparameters**

Hyperparameters are parameters of ML that are not set or changed by the algorithm itself, but by the user to control how the algorithm learns. Here are the hyperparameters we must define:

**learning_rate**: this corresponds to _alpha_ in the Bellman equation and takes on a value from 0 to 1. This represents how much we want the algorithm to change in response to new information

**discount_rate**: this corresponds to _gamma_ in the Bellman equation and takes on a value from 0 to 1. This represents how much we value information from predicted future states

**epsilon**: as we've mentioned before, this is the threshold for choosing to explore or exploit. The hyperparameter variable represents the initial value

**decay_rate**: this is how much we want epsilon to decrease by in every episode. We want to keep decreasing epsilon over time so that the agent is more likely to exploit when it has gathered more information.

**num_episodes**: number of episodes we want to train for

**max_epochs**: the maximum number of epochs or steps we want in each episode. The episode stops either when this threshold is reached, or the passenger is successfully delivered.

**Bonus**: Once you've finished the assignment, come back and change the hyperparameters and see how it affects the algorithm's performance. What values yield the most effective algorithm?

In [ ]:
# hyperparameters
learning_rate = 0.9
discount_rate = 0.8
epsilon = 1.0
decay_rate= 0.005
num_episodes = 1000
max_epochs = 99 # per episode   

## **Training** 

Now we will implement the Q Learning algorithm in the block below! Let's walk through the code step by step.
First, in Block A, we are initializing some lists so we can collect and plot data to analyze our training results. Next, we have a loop for Block B to allow us to repeat the algorithm for each episode. Then we begin each episode by resetting the environment to a new starting position in Block B1. 

**Why do you think we have to set done equal to False?**

Answer:

In Block B3, we have a loop for each episode to go through the individual epochs. In this loop, we must first decide whether we want to explore or exploit in Block B3a.

**Which Case (1 or 2) corresponds to exploring, and which corresponds to exploiting?** *HINT: .sample() picks a random value from the given space, and np.argmax() returns the index of the maximum value in an array*

Answer:

After we decide whether to explore or exploit, we actually perform that action (an integer between 0 and 5), through the step function in Block B3b.

Then, in Block B3c, it's time to update our Q Table based on the reward and new state that results from our action. This is where we finally implement the Bellman Equation. Compare this line of code with the equation presenting at the top of the notebook. Do they match up?

In Blocks B3e and B3f we update our state to the new state that was created with the step function and check to see if the passenger has been successfully delivered, becuase if they have, we can end the episode. But if they haven't, we go back to the start of the inner loop and repeat.

Now we're out of the inner loop and into the outer loop again. All that's left to do for our algorithm is to decrease the value of epsilon by applying the decay rate in Block B4. 

**Does a higher decay rate cause epsilon to decrease faster or slower?**

Answer:

**Note that the training process for Q Learning is analgous to training a neural network. However, instead of updating the weights of nodes, we are updating values in the Q Table.**

In [ ]:
# (Block A) algorithm stats
epochs_list = []
failed_dropoffs_list = []
rewards_list = [] 

# (Block B) Q Learning Algorithm
for episode in range(num_episodes):

    # (Block B1) reset the environment to a random starting point
    state = env.reset()[0]
    done = False
    
    # (Block B2) stats for algorithm performance
    epochs = 0
    failed_dropoffs = 0
    cumulative_reward = 0

    # (Block B3) Run Episode
    for s in range(max_epochs):

        # (Block B3a) exploration-exploitation tradeoff
        if random.uniform(0,1) < epsilon:
            # Case 1
            action = env.action_space.sample()
        else:
            # Case 2
            action = np.argmax(qtable[state,:])

        # (Block B3b)take action and observe new state and reward
        new_state, reward, done, info, _ = env.step(action)

        # (Block B3c) Updating the Q Table (Bellman Equation)
        qtable[state,action] = qtable[state,action] + learning_rate * (reward + discount_rate * np.max(qtable[new_state,:])-qtable[state,action])

        # (Block B3d) update stats
        epochs += 1
        if reward == -10:
            failed_dropoffs += 1
        cumulative_reward += reward
        
        # (Block B3e) Update to our new state
        state = new_state

        # (Block B3f) if done, finish episode
        if done == True:
            break # exit the loop
            
    # (Block B4) Decrease epsilon
    epsilon = np.exp(-decay_rate*episode)
    
    # (Block B5) Update stats lists
    epochs_list.append(epochs)
    failed_dropoffs_list.append(failed_dropoffs)
    rewards_list.append(cumulative_reward)
    
print("Training Complete!")
    

## **Checking Algorithm Performance**

Now that we've trained the algorithm, let's see how our model performed. One way of checking, is by plotting the stats we collected from training. Running the code below will do that for you.

The first plot graphs the total reward for each episode, the second plot graphs the number of epochs per episode, and the third graph plots the number of failed/invalid dropoffs per episode.

In [ ]:
# Plot reward convergence
plt.title("Cumulative reward per episode")
plt.xlabel("Episode")
plt.ylabel("Cumulative reward")
plt.plot(rewards_list)
plt.show()

# Plot epoch convergence
plt.title("# Epochs per episode")
plt.xlabel("Episode")
plt.ylabel("# Epochs")
plt.plot(epochs_list)
plt.show()

# Plot failed dropoff convergence
plt.title("# Failed Dropoffs per episode")
plt.xlabel("Episode")
plt.ylabel("# Failed Dropoffs")
plt.plot(failed_dropoffs_list)
plt.show()

If your first plot is increasing (should look something like the graph of log(x)) and your second two are decreasing (should look like exponential decay), then ***congratulations!*** That means the algorithm is learning. Obviously, if the number of failed dropoffs is decreasing over time, that's a good thing, as well as if the total rewards are increasing over time.

**Why is a decrease in the number of epochs per episode a good thing?** *HINT: what determines the number of epochs in an episode?*

Answer:

Now comes the fun part. Let's see our agent in action by running the code block below. The block below runs through one episode of the Q Learning algorithm, but only exploits what is already known from the Q table. (There is also some extra code to help us create the animation, so don't worry about trying to understand that)

In [ ]:
"""Test algorithm performance after training"""

num_epochs = 0
total_failed_deliveries = 0
experience_buffer = []
store_gif = True

# Initialize experience buffer for animation
my_env = env.reset()
state = my_env[0]
epoch = 1 
done = False

while not done:
    action = np.argmax(qtable[state])
    state, reward, done, _, _ = env.step(action)

    if reward == -10:
        total_failed_deliveries += 1

    # Store rendered frame in animation dictionary
    experience_buffer.append({
        'frame': env.render(),
        'episode': episode,
        'epoch': epoch,
        'state': state,
        'action': action,
        'reward': cum_reward
        }
    )

    num_epochs += 1

if store_gif:
    store_episode_as_gif(experience_buffer)

# Run animation and print output
run_animation(experience_buffer)

# Print final results
print(f"Test results")
print(f"# of steps (epochs): {num_epochs}")
print(f"# failed drop-offs: {total_failed_deliveries}")

### Compare this new animation with the random one from the beginning. Hopefully, you'll see that the taxi drives essentially perfectly!

Note that we never had to explain the actual task in code, yet the agent was able to learn it perfectly. Hopefully, this example has shown you what a powerful tool reinforcement learning is.

If you're curious, here are a few changes you can make to see how performance is affected:
- adjust hyperparameters and number of episodes and epochs
- implement action mask (a feature of gym that only allows the agent to choose actions that will make a change in the environment)
- implement randomized tie-breaking during exploitation step
- override reward function
- use a different RL algorithm

## **Sources**

https://gymnasium.farama.org/environments/toy_text/taxi/ \
https://towardsdatascience.com/solving-the-taxi-environment-with-q-learning-a-tutorial-c76c22fc5d8f\
https://medium.com/swlh/introduction-to-q-learning-with-openai-gym-2d794da10f3d \
https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/ \
https://www.gocoder.one/blog/rl-tutorial-with-openai-gym/ 

Created by Aakarsh Vermani
